In [ ]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:140% !important; }</style>"))

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://github.com/philhoonoh/blog_git/blob/main/comp_models_1.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View Source</a>
  </td>
</table>

# PyTorch Model - timm library, torchvision.models part1)
  
> Basic usage of timm and torchvision.model libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from torchvision import models

## Timm Library

#### Select Model
```python
timm.list_models('model name or regular exp', pretrained = True)
```

In [2]:
all_densenet_models = timm.list_models('*convnext*', pretrained = True)
all_densenet_models

['convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k']

#### Check Model Architecture

```python
model = timm.create_model('convnext_small', pretrained=True)
print(model)
```

In [3]:
model = timm.create_model('convnext_small', pretrained=True)
# print(model)

#### Change the last layer of the Model
```python
model = timm.create_model('convnext_small', pretrained=True, num_classes = 100)
```

In [4]:
model = timm.create_model('convnext_small', pretrained=True, num_classes = 100)
# print(model2)

####  Testing 

```python
model = timm.create_model('convnext_small', pretrained=True, num_classes = 100)
model.eval()
result = model(torch.randn(1,3,224,224))
print(result)
print(result.shape)
```

In [5]:
model = timm.create_model('convnext_small', pretrained=True, num_classes = 100)
model.eval()
result = model(torch.randn(1,3,224,224))
print(result)
print(result.shape)

tensor([[-0.0158,  0.1046,  0.0368, -0.0476,  0.0025, -0.0348, -0.0098,  0.0188,
          0.0295,  0.0457,  0.0469,  0.0639, -0.0396, -0.0117, -0.0285, -0.0227,
          0.0458, -0.0168, -0.0154, -0.0092,  0.0503, -0.0025, -0.0331,  0.0323,
         -0.0633, -0.0273,  0.0145, -0.1236, -0.0766,  0.0006,  0.0107,  0.0774,
          0.0476, -0.0875, -0.0372,  0.1161, -0.0769, -0.0529,  0.0034, -0.0750,
         -0.0091,  0.0050, -0.0225, -0.0374,  0.0078, -0.0454,  0.0190,  0.0216,
         -0.0353,  0.0156, -0.0068, -0.0046,  0.0382, -0.0386,  0.0546,  0.1143,
         -0.1125,  0.0425, -0.0689,  0.0880, -0.0387,  0.0296,  0.0119, -0.0203,
         -0.0192,  0.0312, -0.0255,  0.0025,  0.0131, -0.0624, -0.0700, -0.0135,
         -0.0142,  0.0672,  0.0155, -0.0372, -0.0647,  0.0303,  0.0476, -0.0135,
          0.0137, -0.0134, -0.0938,  0.0759, -0.0036, -0.0033,  0.0143,  0.0158,
          0.0498, -0.0079, -0.0516, -0.0229,  0.0904, -0.0091,  0.0702, -0.0282,
          0.0008, -0.0302, -

#### Custom Model using convnext_small

> Implementing Custom Model for ThreeWayStratifiedDataset described in [https://psyduck5.tistory.com/10]  
> Attempt1. Sharing pretraine embeddings but constructing differnet layers for each labels (age, mask, gender)  
> Attempt2. For Generalization, applying dropout layers  

In [6]:
class ConvNextLSmallCustom(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model('convnext_small', pretrained=True, num_classes = 1536)
        self.dropout = nn.Dropout(0.5)
        self.dropouts = nn.ModuleList([
                    nn.Dropout(0.5) for _ in range(5)])
        self.age_layer = nn.Linear(in_features=1536, out_features=3, bias=True)
        self.mask_layer = nn.Linear(in_features=1536, out_features=2, bias=True)
        self.sex_layer = nn.Linear(in_features=1536, out_features=2, bias=True)
    
    def forward(self, x):
        x = self.model(x)
        x_ = self.dropout(x)
        
        for i, dropout in enumerate(self.dropouts):
            if i==0:
                x_age = self.age_layer(dropout(x_))
                x_mask = self.mask_layer(dropout(x_))
                x_sex = self.sex_layer(dropout(x_))
            else:
                x_age += self.age_layer(dropout(x_))
                x_mask = self.mask_layer(dropout(x_))
                x_sex = self.sex_layer(dropout(x_))
        else:
            x_age /= len(self.dropouts)
            x_mask /= len(self.dropouts)
            x_sex /= len(self.dropouts)
        
        return x_age, x_mask, x_sex

## Torchvision models

#### Basic Template for Model

> applicable any libraries based on pytorch

```python
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        pass

    def forward(self, x):
        pass
```

#### Select Model

> Check out the list of models on https://pytorch.org/vision/master/models.html

#### Check Model Architecture
 
```python
model = models.densenet161(pretrained = True)
print(model)
```

In [7]:
model = models.densenet161(pretrained = True)
# print(model)

#### Change the last layer of the Model

> Needs to manually check out the name of last layer : usually either 'fc' or 'classifier'  

```python
num_classes = 100
model.classifier = nn.Linear(in_features = model.classifier.in_features,
                                      out_features = num_classes, bias = True)
print(model)
```

In [8]:
num_classes = 100
model.classifier = nn.Linear(in_features = model.classifier.in_features,
                                      out_features = num_classes, bias = True)
# print(model)

####  Testing 

```python
model = models.densenet161(pretrained = True))
num_classes = 100
model.classifier = nn.Linear(in_features = model.classifier.in_features,
                                      out_features = num_classes, bias = True)
model.eval()
result = model(torch.randn(1,3,224,224))
print(result)
print(result.shape)
```

In [9]:
model = models.densenet161(pretrained = True)
num_classes = 100
model.classifier = nn.Linear(in_features = model.classifier.in_features,
                                      out_features = num_classes, bias = True)
model.eval()
result = model(torch.randn(1,3,224,224))
print(result)
print(result.shape)

tensor([[-0.2660,  0.4000,  0.0566,  0.7332,  0.0120,  0.0402,  0.0670, -1.0042,
          0.3829, -0.0477, -0.4843, -0.0874,  0.0382, -0.2041,  0.5817,  0.1364,
          0.3040, -0.5408, -0.2544,  0.2450, -0.0206,  0.6274,  0.2964,  0.4478,
         -0.3061,  0.0265,  0.2741,  0.2010, -0.2461, -0.2156,  0.4659, -0.0693,
          0.0226, -0.3524,  0.8735,  0.0297, -0.2518, -0.0303, -0.7970, -0.3719,
          0.1582,  0.3348, -0.1031,  0.4719, -0.4797, -0.7944,  0.3459,  0.5960,
         -0.0273,  0.3393,  0.2273, -0.6286, -0.2468, -0.0935, -0.0861,  0.3848,
         -0.4297,  0.2101, -0.7249,  0.3109,  0.2408,  0.1554, -0.3098, -0.1568,
         -0.2809, -0.5357,  0.0345,  0.0989, -0.4031, -0.0775, -0.6316,  0.7590,
          0.0898,  0.0369, -0.3507, -0.6202,  0.9339, -0.0096, -0.6873, -0.2853,
          0.4350, -0.4197,  0.3255,  0.1393,  0.3737,  0.2953, -0.1338, -0.5609,
          0.2145,  0.8751, -0.7551,  0.7100, -0.1903, -0.2114,  0.3616,  0.3200,
         -0.5430, -0.3831, -

#### Initialzing with functions 

> Implementing user-defined functions for changing the last layer and initializing weights  
> Attempt1. Define a function to automatically finds out the last layer of the model and change the out_features  
> Attempt2. Define a function to initialzing wegihts to the modified layers.  
> With these functions, anyone can easily contruct the custom model with different out_features just like timm library

In [11]:
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_uniform_(m.weight.data, nonlinearity='relu') 
    
def change_last_layer(model, num_classes):
    name_last_layer = list(model.named_modules())[-1][0]
    
    if name_last_layer == 'classifier':
        model.classifier = nn.Linear(in_features = model.classifier.in_features,
                                              out_features = num_classes, bias = True)
        initialize_weights(model.classifier)
        return model
    
    elif name_last_layer == 'fc':
        model.fc = nn.Linear(in_features = model.fc.in_features,
                                      out_features = num_classes, bias = True)
        initialize_weights(model.fc)
        return model
    
    else:
        raise Exceptionception('last layer should be nn.Linear Module named as either fc or classifier')

In [12]:
class DenseNet161(nn.Module):
    def __init__(self, num_classes):
        super(DenseNet161, self).__init__()
        self.model = models.densenet161(pretrained = True)
        self.model = change_last_layer(self.model, num_classes)
    
    def forward(self, x):
        x = self.model(x)
        return x

In [13]:
model = DenseNet161(10)
model.eval()
model(torch.randn(1,3,224,224))

tensor([[ 1.1248, -0.3958, -0.0994,  0.2717,  0.2535, -0.4034, -0.0367,  0.4897,
          0.8369,  0.4912]], grad_fn=<AddmmBackward0>)